# Profiling
### Finding bottlenecks in your Python program

<img src="https://www.explainxkcd.com/wiki/images/e/e2/estimating_time.png" style="width: 500px;"/>


## Contents

* Timing
* Profiling
* Line-by-line profiling

**"First make it work. Then make it right. Then make it fast."**
 
-  Kent Beck

Prior to code optimization, hotspots and bottlenecks must be located. 

## Profiling allows us to measure resources used by sections of the program. 

Typical resources of interest are
* Amount of CPU used
* Amount of RAM used

But also other resources can be measured, such as:

* Disk I/O
* Network I/O
* ...

Today, we only consider CPU profiling.

## Profiling techniques

Start simple, switch to more complex techniques if needed!

Techniques for measuring CPU time in increasing complexity:
1. Manual timing
2. The `timeit` module
3. The `cprofile` module

## Case study: filling a grid with point values

* Consider a rectangular 2D grid



<center> 
    <img src="pdf/grid_lattice.svg" style="width: 200px;"/>
Grid lattice
</center>

* A NumPy array `a[i,j]` holds values at the grid points

# Implementation

In [16]:
from numpy import *

class Grid2D(object):
    def __init__(self,
                 xmin=0, xmax=1, dx=0.5,
                 ymin=0, ymax=1, dy=0.5):
        
        self.xcoor = arange(xmin, xmax+dx, step=dx)
        self.ycoor = arange(ymin, ymax+dy, step=dy)

    def gridloop(self, f):
        lx = size(self.xcoor)
        ly = size(self.ycoor)
        a = zeros((lx,ly))

        for i in range(lx):
            x = self.xcoor[i]
            for j in range(ly):
                y = self.ycoor[j]
                a[i,j] = f(x, y)
        return a

# Usage

Create a new grid:

In [17]:
g = Grid2D(dx=0.001, dy=0.001)

Computing grid values:

In [19]:
import numpy

def myfunc(x, y):
    return numpy.sin(x*y) + y

print("Computing values...")
a = g.gridloop(myfunc)
print("done")

Computing values...
done


# Timing

Use `time.time()` to measure the time spend in a code section.
  ```python
  t0 = time.time()
  # execute code here
  t1 = time.time()
  print("Runtime: {}".format(t1-t0))
  ```
  

Timing guidelines:
* Put simple statements in a loop.
* Make sure to use a constant machine load.
* Run the tests several times, choose the **smallest**. Why?

*Note*: 
* time.time() measures **wall clock time**. 
* time.clock() measures **CPU time**.

## Timing of the case study

The case study has two parts that could potentially be slow: 
1. The initialisation `Grid2D(dx=0.001, dy=0.001)`
2. Calling the `g.gridloop(myfunc)` function.

We time these two parts separately to figure out how much time is spend in each.

### Timing the Grid2D initialisation

In [20]:
import time

for i in range(1, 4):
    t0 = time.time()
    g = Grid2D(dx=0.001, dy=0.001)
    t1 = time.time()
    print("Experiment {}, CPU time: {:.4} s".format(i, t1-t0))
print("Done")

Experiment 1, CPU time: 4.292e-05 s
Experiment 2, CPU time: 0.0002937 s
Experiment 3, CPU time: 2.789e-05 s
Done


### Timing the `gridloop` function

In [23]:

import time
print(time.time())

#for i in range(1, 4):
#    t0 = time.time()
#    g.gridloop(myfunc)
#    t1 = time.time()
#    print("Experiment {}. CPU time: {:.4} s".format(i, t1-t0))
#print("Done")

1569401457.8328924


$=>$ The gridloop function is causing the slow execution!

## The *timeit* module

The `timeit` module provides an convenient way for measuring the CPU time of small code snippets.

Usage:

In [24]:
import timeit

timeit.timeit(stmt="a+=1", setup="a=0")

0.06533873200169182

The code is automatically wrapped in a for loop. By default the command is executed 1,000,000 times. It returns the **accumulated** runtime.

You can adjust the the number of executions:

In [7]:
timeit.timeit(stmt="a+=1",setup="a=0", number=10000)

0.000572180997551186

Use timeit.repeat if you would like to repeat the experiment multiple times:

In [8]:
timeit.repeat(stmt="a+=1",setup="a=0", number=10000, repeat=5)

[0.0006758550007361919,
 0.0009771650002221577,
 0.0008011019999685232,
 0.0005401749986049253,
 0.0006059890001779422]

## Timing user defined function

Timeit creates its own namespace - which means that variables, functions, ... are not avaible in timeit, unless they are imported in the setup argument:

In [9]:
timeit.repeat(stmt="g.gridloop(myfunc)", setup="from __main__  import g, myfunc", repeat=5, number=1)

[3.2948365679985727,
 2.8746060699995724,
 2.6881581689995073,
 2.874211877999187,
 2.921613537000667]

## Profiling modules with cProfile

A profile is a set of statistics that describes how often and for how long various parts of the program executed.

`cProfile` is two main (deterministic) profiling module in Python.

### Using cProfile

* As a script: `python -m cProfile script.py
`

* As a module:

In [10]:
import cProfile
pr = cProfile.Profile()
res = pr.run("g.gridloop(myfunc)")  # res contains the statistics
res.print_stats()                   # Print profiling statistics 

         1002008 function calls in 2.988 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.642    0.642    2.988    2.988 <ipython-input-1-bdd39b4c44e8>:11(gridloop)
  1002001    2.345    0.000    2.345    0.000 <ipython-input-3-a51c4e6c8393>:3(myfunc)
        1    0.000    0.000    2.988    2.988 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:2889(size)
        1    0.000    0.000    2.988    2.988 {built-in method builtins.exec}
        1    0.001    0.001    0.001    0.001 {built-in method numpy.zeros}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




## Getting runtime statistics

Header definitions
* **ncalls**: number of calls
* **tottime**: total time spent in the given function excluding time made in calls to sub-functions
* **percall**: tottime divided by ncalls
* **cumtime**: cumulative time spent in this and all subfunctions
* **percall**: cumtime divided by ncalls
* **filename:lineno(function)**: information about the function

Tip: You can also dump the profiling data to disk and use the `pstats` Python module to get more control over the statistics (sort, restrict, ...).

## Back to our case-study: What have we learned so far?

The biggest contributors of the total runtime are:
   1. `gridloop` contributes one third of the total runtime.
   2. `myfunc` calls contributes two thirds of the total runtime.

* `myfunc` is fairly straight-forward
  ```python
  def myfunc(x, y):
    return sin(x*y) + y
  ```
  Might be difficult to improve.
* What about `gridloop`?

## Recall that, `gridloop` was a function of the form

```python
def gridloop(self, f):
    lx = size(self.xcoor)
    ly = size(self.ycoor)
    a = zeros((lx,ly))

    for i in range(lx):
        x = self.xcoor[i]
        for j in range(ly):
            y = self.ycoor[j]
             a[i,j] = f(x, y)
    return a
```

It would be useful to see how much time is spend in each line!

## Line by line profiling

The line_profiler module inspects the time spend in each line of a Python function. 

## Usage

1. Install with `conda install line_profiler`
2. "Decorate" the function of interest with `@profile`:
    ```python
    @profile
    def gridloop(func):
        # ...
    ```
3. Run line profiler with:
    ```bash
    kernprof -l -v grid2d_lineprofile.py
    ```

## Demo

In [25]:
!kernprof -l -v grid2d_lineprofile.py

Wrote profile results to grid2d_lineprofile.py.lprof
Timer unit: 1e-06 s

Total time: 4.63776 s
File: grid2d_lineprofile.py
Function: gridloop at line 11

Line #      Hits         Time  Per Hit   % Time  Line Contents
    11                                               @profile
    12                                               def gridloop(self, f):
    13         1          7.0      7.0      0.0          lx = size(self.xcoor)
    14         1          2.0      2.0      0.0          ly = size(self.ycoor)
    15         1         18.0     18.0      0.0          a = zeros((lx,ly))
    16                                           
    17      1002        593.0      0.6      0.0          for i in range(lx):
    18      1001        859.0      0.9      0.0              x = self.xcoor[i]
    19   1003002     622880.0      0.6     13.4              for j in range(ly):
    20   1002001     740304.0      0.7     16.0                  y = self.ycoor[j]
    21   1002001    3273095.0      3.3  

**Conclusion:** A significant amount of time is spend in loops and indexing. How can we improve this?

## A vectorised Grid2D implementation

In [26]:
class VectorisedGrid2D(object):
    def __init__(self,
                 xmin=0, xmax=1, dx=0.5,
                 ymin=0, ymax=1, dy=0.5):
        
        self.xcoor = arange(xmin, xmax+dx, step=dx)
        self.ycoor = arange(ymin, ymax+dy, step=dy)

    def gridloop(self, f):
        return f(self.xcoor[:,None], self.ycoor[None,:])  # Vectorized grid evaluation 

## Timing the vectorised Grid2D

In [30]:
vg = VectorisedGrid2D(dx=0.001, dy=0.001)
min(timeit.repeat(stmt="vg.gridloop(myfunc)", setup="from __main__  import vg, myfunc", repeat=5, number=1))

0.016483209998114035

In [31]:
g = Grid2D(dx=0.001, dy=0.001)
min(timeit.repeat(stmt="g.gridloop(myfunc)", setup="from __main__  import g, myfunc", repeat=5, number=1))

2.6357826870007557

**Vectorization yields a ca 70 times speed improvement!**